In [2]:
import os
os.chdir('../')
%pwd

'd:\\Projects\\NLP\\Text-Summarizer-Project'

In [3]:
%pwd

'd:\\Projects\\NLP\\Text-Summarizer-Project'

In [4]:
#create entity for dataclass

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class ModelTrainerConfig:
    root_dir : Path
    data_path : Path
    model_ckpt : Path
    num_train_epochs : int
    warmup_steps : int
    per_device_train_batch_size : int
    weight_decay : float
    logging_steps : int
    evaluation_strategy : str
    eval_steps : int
    save_steps : float
    gradient_accumulation_steps : int

In [5]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        #create a artifacts folder\
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self)-> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments
        
        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            data_path= config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [7]:
from textSummarizer.logging import logger
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import TrainingArguments, Trainer
from datasets import load_dataset,load_from_disk
import torch

c:\Users\senthil\anaconda3\envs\testsum\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)

        #loading data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        training_args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps= self.config.warmup_steps,
            per_device_train_batch_size= self.config.per_device_train_batch_size,
            per_device_eval_batch_size= self.config.per_device_train_batch_size,
            weight_decay= self.config.weight_decay,
            logging_steps= self.config.logging_steps,
            evaluation_strategy= self.config.evaluation_strategy,
            eval_steps= self.config.eval_steps,
            save_steps= self.config.save_steps,
            gradient_accumulation_steps= self.config.gradient_accumulation_steps
        )

        trainer = Trainer(
            model= model_pegasus,
            args= training_args,
            tokenizer= tokenizer,
            data_collator= seq2seq_data_collator,
            train_dataset= dataset_samsum_pt['test'],
            eval_dataset= dataset_samsum_pt['validation']
        )

        trainer.train()

        #save model 
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,'pegasus-samsum-model'))

        #save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,'tokenizer'))



In [10]:
#create the pipeline

try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
    


except Exception as e:
    raise e


[2023-09-21 16:02:27,896: INFO: common: yaml file : config\config.yaml loaded sucessfully]
[2023-09-21 16:02:27,947: INFO: common: yaml file : params.yaml loaded sucessfully]
[2023-09-21 16:02:27,955: INFO: common: created directory at : artifacts]
[2023-09-21 16:02:27,965: INFO: common: created directory at : artifacts/model_trainer]


ConnectionError: HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /google/pegasus-cnn_dailymail/resolve/main/tf_model.h5 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001527AAD0FD0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))